In [55]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc

In [2]:
segment_count = pd.read_csv("./data/customersegmentcount.csv", names=['cust_segment', 'count'])
segment_count.head()

,cust_segment,count
0,45-54|Female|Middle|Baby|Unknown|Married,3
1,Unknown|Unknown|VeryHigh|Youth|House|Separated,6
2,18-24|Male|High|None|Flat|Married,3
3,25-34|Female|Low|None|Unknown|Cohabiting,10
4,35-44|Unknown|High|Baby|Unknown|Separated,4


In [3]:
segment_count.cust_segment.nunique() ## all unique categories

5064

In [4]:
segment_count[["Age", "Gender", "Income", "Children", "Housing", "Relationship"]] = segment_count.cust_segment.str.split("|", expand=True)
segment_count = segment_count.iloc[:, [2,3,4,5,6,7,1]]
segment_count

,Age,Gender,Income,Children,Housing,Relationship,count
0,45-54,Female,Middle,Baby,Unknown,Married,3
1,Unknown,Unknown,VeryHigh,Youth,House,Separated,6
2,18-24,Male,High,None,Flat,Married,3
3,25-34,Female,Low,None,Unknown,Cohabiting,10
4,35-44,Unknown,High,Baby,Unknown,Separated,4
...,...,...,...,...,...,...,...
5059,25-34,Male,Middle,Preschool,Unknown,Separated,1
5060,35-44,Female,High,Preschool,Unknown,Unknown,1
5061,Unknown,Male,Middle,Teenager,Unknown,Married,1
5062,18-24,Female,VeryHigh,Preschool,House,Cohabiting,1


In [60]:
segment_count.to_pickle("segment_count.pickle")

In [5]:
segment_revenue = pd.read_csv("./data/customersubcatgrossrevenuebysegment.csv", names=['cust_segment', "purchase_category", 'revenue'])
segment_purchase_count = pd.read_csv("./data/customersubcatpurchasecountbysegment.csv", names=['cust_segment', "purchase_category", 'total_purchases'])

In [6]:
segment_revenue.head()

,cust_segment,purchase_category,revenue
0,35-44|Male|Middle|Youth|Flat|Married,Home|Beds & Mattresses,733.450000
1,Unknown|Female|Low|Youth|House|Unknown,Home|Beds & Mattresses,10745.972000
2,Unknown|Unknown|Low|Teenager|Flat|Married,Garden|Garden Leisure,136.001000
3,35-44|Male|Middle|Teenager|House|Married,Healthcare|Slimming & Detox Supplements,27.980000
4,25-34|Unknown|Middle|Preschool|Flat|Single,Home|Bathroom,56.688668


In [7]:
segment_purchase_count.head()

,cust_segment,purchase_category,total_purchases
0,35-44|Female|High|Teenager|House|Cohabiting,Garden|Garden Tools,22
1,25-34|Female|Middle|None|House|Married,Garden|Garden Tools,149
2,Unknown|Female|Unknown|Unknown|Flat|Unknown,Garden|Garden Tools,153
3,Unknown|Female|Middle|None|House|Unknown,Pets|Care,21
4,Unknown|Unknown|Middle|Teenager|House|Married,Learning|Financial,16


In [8]:
segment_revenue.cust_segment.nunique(), segment_revenue.purchase_category.nunique()

(5064, 375)

In [9]:
segment_purchase_count.cust_segment.nunique(), segment_purchase_count.purchase_category.nunique()

(5064, 375)

In [10]:
## merge the two dataframes
segment_revenue_and_count = pd.merge(segment_revenue, 
                                     segment_purchase_count,
                                     how = "outer", 
                                     on = ["cust_segment", "purchase_category"], 
                                     indicator=True)

segment_revenue_and_count.head()

,cust_segment,purchase_category,revenue,total_purchases,_merge
0,35-44|Male|Middle|Youth|Flat|Married,Home|Beds & Mattresses,733.450000,6.0,both
1,Unknown|Female|Low|Youth|House|Unknown,Home|Beds & Mattresses,10745.972000,67.0,both
2,Unknown|Unknown|Low|Teenager|Flat|Married,Garden|Garden Leisure,136.001000,4.0,both
3,35-44|Male|Middle|Teenager|House|Married,Healthcare|Slimming & Detox Supplements,27.980000,1.0,both
4,25-34|Unknown|Middle|Preschool|Flat|Single,Home|Bathroom,56.688668,7.0,both


In [56]:
del segment_revenue
del segment_purchase_count
gc.collect()

9612

In [11]:
segment_revenue_and_count._merge.value_counts()

both          356215
left_only        743
right_only         0
Name: _merge, dtype: int64

In [12]:
## there are missing values in total_purchases
segment_revenue_and_count[segment_revenue_and_count._merge == "left_only"].head()

,cust_segment,purchase_category,revenue,total_purchases,_merge
226,35-44|Male|Low|Teenager|House|Separated,Healthcare|Quit Smoking,9.475,NaN,left_only
1812,35-44|Female|High|Preschool|Flat|Single,Fitness|Sportswear,7.990,NaN,left_only
7592,65+|Female|Middle|Teenager|Unknown|Cohabiting,Garden|Plants & Flowers,59.151,NaN,left_only
9189,25-34|Female|High|None|House|Separated,Spas & Country House|Attractions,409.650,NaN,left_only
9192,18-24|Female|Low|None|House|Separated,Beach Holidays|Greece,10.000,NaN,left_only


In [13]:
## As the total_purchases have NaNs, a good measure could be to determine 
## the average ticket size of a purchase_category and filling the missing
## value with the approximate total_purchases that will generate the corresponding revenue.

In [14]:
## find avg ticket size for _purchase_category
avg_ticket_size_by_purchase_category = (segment_revenue_and_count[segment_revenue_and_count._merge != "left_only"]
                                        .groupby("purchase_category")
                                        .apply(lambda g: sum(g.revenue)/sum(g.total_purchases))
                                        .reset_index()
                                        .rename(columns = {0:"avg_revenue"}))
avg_ticket_size_by_purchase_category

,purchase_category,avg_revenue
0,Activities|Adventure & Theme Parks & Zoos,20.149809
1,Activities|Animals,24.765165
2,Activities|Archery/Shooting,25.557389
3,Activities|Ballooning,306.325000
4,Activities|Dance,12.210471
...,...,...
370,UK Seaside|Other Seaside,115.307378
371,Wedding|Bridal Beauty,14.662469
372,Wedding|Flowers,7.016250
373,Wedding|Photography,171.625000


In [15]:
## join it back to the segment_revenue_and_count dataframe
segment_revenue_and_count = pd.merge(segment_revenue_and_count,
                                     avg_ticket_size_by_purchase_category,
                                     on = "purchase_category")

In [16]:
## fill the missing value with the number of transactions

segment_revenue_and_count.loc[segment_revenue_and_count._merge == "left_only", "total_purchases"] = np.ceil(segment_revenue_and_count.loc[segment_revenue_and_count._merge == "left_only", "revenue"]/segment_revenue_and_count.loc[segment_revenue_and_count._merge == "left_only", "avg_revenue"])
segment_revenue_and_count["total_purchases"] = segment_revenue_and_count.total_purchases.astype(int)
segment_revenue_and_count.head()

<IPython.core.display.Javascript object>

,cust_segment,purchase_category,revenue,total_purchases,_merge,avg_revenue
0,35-44|Male|Middle|Youth|Flat|Married,Home|Beds & Mattresses,733.450,6,both,165.608162
1,Unknown|Female|Low|Youth|House|Unknown,Home|Beds & Mattresses,10745.972,67,both,165.608162
2,45-54|Unknown|VeryHigh|None|House|Separated,Home|Beds & Mattresses,2204.830,16,both,165.608162
3,25-34|Unknown|Middle|Teenager|House|Married,Home|Beds & Mattresses,25627.180,145,both,165.608162
4,35-44|Unknown|Middle|None|Flat|Unknown,Home|Beds & Mattresses,3405.240,21,both,165.608162


In [17]:
## drop columns not needed now

segment_revenue_and_count.drop(columns = ["_merge", "avg_revenue"], inplace = True)
segment_revenue_and_count.head()

,cust_segment,purchase_category,revenue,total_purchases
0,35-44|Male|Middle|Youth|Flat|Married,Home|Beds & Mattresses,733.450,6
1,Unknown|Female|Low|Youth|House|Unknown,Home|Beds & Mattresses,10745.972,67
2,45-54|Unknown|VeryHigh|None|House|Separated,Home|Beds & Mattresses,2204.830,16
3,25-34|Unknown|Middle|Teenager|House|Married,Home|Beds & Mattresses,25627.180,145
4,35-44|Unknown|Middle|None|Flat|Unknown,Home|Beds & Mattresses,3405.240,21


In [18]:
## Split the cust_segment and purchase_category to separater columns

In [19]:
segment_revenue_and_count[["Age", "Gender", "Income", "Children", "Housing", "Relationship"]] = segment_revenue_and_count.cust_segment.str.split("|", expand=True)
segment_revenue_and_count[["PurchaseCategoryA", "PurchaseCategoryB"]] = segment_revenue_and_count.purchase_category.str.split("|", expand=True)

segment_revenue_and_count = segment_revenue_and_count.iloc[:, [4,5,6,7,8,9,10,11,2,3]]
segment_revenue_and_count.head()

,Age,Gender,Income,Children,Housing,Relationship,PurchaseCategoryA,PurchaseCategoryB,revenue,total_purchases
0,35-44,Male,Middle,Youth,Flat,Married,Home,Beds & Mattresses,733.450,6
1,Unknown,Female,Low,Youth,House,Unknown,Home,Beds & Mattresses,10745.972,67
2,45-54,Unknown,VeryHigh,None,House,Separated,Home,Beds & Mattresses,2204.830,16
3,25-34,Unknown,Middle,Teenager,House,Married,Home,Beds & Mattresses,25627.180,145
4,35-44,Unknown,Middle,None,Flat,Unknown,Home,Beds & Mattresses,3405.240,21


In [46]:
segment_revenue_and_count.shape

(356958, 10)

In [51]:
for col in segment_revenue_and_count.columns[:-2]:
    segment_revenue_and_count[col] = segment_revenue_and_count[col].astype("category")

In [59]:
segment_revenue_and_count.to_pickle("segment_revenue_and_count.pickle")

In [33]:
## segment_revenue_and_count is the final dataframe that we should analyze